# LangChain 에이전트에 대해서 알아보자!!

## 1. 기본설정

### 필요한 페키지 설치

```bash
# 기본 LangChain 라이브러리
pip install langchain langchain-core langchain-community

# Gemini 2.0 통합을 위한 패키지
pip install google-generativeai langchain-google-genai

# Groq 통합을 위한 패키지
pip install groq langchain-groq
```


In [1]:
# 기본 LangChain 라이브러리
!pip install langchain langchain-core langchain-community

# Gemini 2.0 통합을 위한 패키지
!pip install google-generativeai langchain-google-genai

# Groq 통합을 위한 패키지
!pip install groq langchain-groq

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 24.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------- ----------------------- 1.0/2.5 MB 4.6 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 4.5 MB/s eta 0:00:00
  Using cached google_auth-2.38.0-py2.py3-none-any.whl.metadata (4.8 kB)
INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached protobuf-5.29.3-cp310-abi3-win_am

# 🕵️‍♂️ Agent 생성

## Agent가 뭔데?
내가 아는 Agent는 **스파이 패밀리**의 Agent 황혼 밖에 없는데?? 🤔  

<div style="text-align: center;">
    <img src="https://i.namu.wiki/i/QUelTFbRO2vwJ9d-aVdZpiYmjVz8YBBoPhXn0pRce0qI5Ta_UDOOtZ_Y4RklUtBB9Hc6VW5t8wWMOVO-x5rNNRSP6LUzulwiOpFrsvEtzga2egY0iTbR0d7wR5Ji837LwwlJ_PfZWNOIfFQroBSZcw.webp" width="30%" height="auto">
</div>

---

## 🔗 LangChain 공식 문서에서의 Agent 정의
> **The core idea of agents is to use a language model to choose a sequence of actions to take.**  
> **In chains, a sequence of actions is hardcoded (in code).**  
> **In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.**  

즉, **Agent는 언어 모델을 활용하여 어떤 행동을 할지, 어떤 순서로 진행할지를 결정하는 역할을 한다.**  
반면, **Chain은 미리 정해진 순서대로 실행되는 코드**라는 차이점이 있다.

### 정리하자면 행동은 Agent 순서는 Chain이다

📌 더 자세한 내용은 [LangChain 공식 문서](https://python.langchain.com/v0.1/docs/modules/agents/)에서 확인할 수 있다.
찡긋~


In [29]:
import os
from langchain.agents import AgentType, initialize_agent, load_tools

# Gemini용 API 키 설정
os.environ["GOOGLE_API_KEY"] = "이재욱_GOOGLE_API_키"

# Groq용 API 키 설정
os.environ["GROQ_API_KEY"] = "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"

# Gemini 모델 설정
from langchain_google_genai import ChatGoogleGenerativeAI
gemini_llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0)

# Groq의 Llama 모델 설정
from langchain_groq import ChatGroq
llama_llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0.7)

## 2. 간단한 에이전트 생성하기
### 다양한 LLM 모델 사용 예시
**Gemini 에이전트**

# Gemini 에이전트 생성
gemini_tools = load_tools(["llm-math"], llm=gemini_llm)
gemini_agent = initialize_agent(
    gemini_tools,
    gemini_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Gemini 에이전트 실행
gemini_result = gemini_agent.run("100 * 4를 계산하고 결과에 25를 더하세요.")
print(f"Gemini 결과: {gemini_result}")

**Llama 에이전트 (Groq 사용)**


In [9]:
# LLMMathChain 사용 방식
from langchain.chains import LLMMathChain

# LLMMathChain 초기화
math_chain = LLMMathChain.from_llm(llama_llm)

# 계산 수행 (한 번에 계산)
try:
    result = math_chain.run("(100 * 4) + 25")
    print(f"Llama 결과: {result}")
except Exception as e:
    print(f"오류 발생: {e}")
    
    # 문제가 있다면 단계별 계산 시도
    try:
        result1 = math_chain.run("100 * 4")
        print(f"중간 계산 결과: {result1}")
        
        result2 = math_chain.run(f"{result1} + 25")
        print(f"최종 결과: {result2}")
    except Exception as e2:
        print(f"단계별 계산에서도 오류 발생: {e2}")

Llama 결과: Answer: 425


In [23]:
from langchain.agents import AgentType, initialize_agent
from langchain.tools import BaseTool
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.schema import SystemMessage
from typing import Dict, List, Optional, Any
from pydantic import BaseModel, Field
import re

# 기존에 설정된 llama_llm(Groq)을 사용합니다.

# 도구 입력 모델 정의
class ScriptGeneratorInput(BaseModel):
    image_description: str = Field(..., description="사용자가 떠올린 이미지나 장면에 대한 묘사")
    highlight_point: str = Field(..., description="친구나 가족에게 소개할 때 강조하고 싶은 부분")
    one_word_expression: str = Field(..., description="그 대상이나 장소를 한 마디로 표현한 것과 이유")
    topic: Optional[str] = Field(None, description="OPIC 질문의 주제 (예: 식당, 여행, 취미 등)")

class ScriptAnalyzerInput(BaseModel):
    script: str = Field(..., description="분석할 OPIC 스크립트")

class ScriptRefinerInput(BaseModel):
    original_script: str = Field(..., description="원본 OPIC 스크립트")
    analysis: str = Field(..., description="스크립트 분석 결과")

# 스크립트 생성 도구
class ScriptGenerator(BaseTool):
    name: str = "script_generator"
    description: str = "사용자 입력을 바탕으로 OPIC IH 수준의 스크립트를 생성합니다."
    args_schema: type[BaseModel] = ScriptGeneratorInput
    
    def _run(self, image_description: str, highlight_point: str, one_word_expression: str, topic: Optional[str] = None) -> str:
        # 프롬프트 구성
        prompt = f"""
        Generate an OPIC IH (Intermediate High) level script based on the following user inputs:

        [User's Image Description]: {image_description}
        [Highlight Point]: {highlight_point}
        [One-word Expression and Reason]: {one_word_expression}
        {f"[Topic]: {topic}" if topic else ""}

        Requirements:
        1. Create 6-7 sentences to fit a 1-1.5 minute speaking time
        2. Use complex sentences rather than simple ones
        3. Include appropriate transition phrases and idiomatic expressions for IH level
        4. Create a cohesive flow using all three inputs
        5. Format the output as a first-person narrative
        6. Include relevant details from the user's inputs

        Output format:
        [SCRIPT]
        (Script content)

        [WORD COUNT]: (word count)
        [ESTIMATED SPEAKING TIME]: (estimated time)
        [COMPLEX SENTENCES]: (number of complex sentences)
        [KEY EXPRESSIONS]: (3-4 key expressions used)
        """
        
        # llama_llm을 사용하여 스크립트 생성
        script = llama_llm.predict(prompt)
        
        return script

# 스크립트 분석 도구
class ScriptAnalyzer(BaseTool):
    name: str = "script_analyzer"
    description: str = "생성된 OPIC 스크립트를 분석하여 개선점을 제시합니다."
    args_schema: type[BaseModel] = ScriptAnalyzerInput
    
    def _run(self, script: str) -> str:
        # 프롬프트 구성
        prompt = f"""
        Analyze the following OPIC script for IH level and provide improvement suggestions:

        {script}

        Please analyze:
        1. Is it appropriate for IH level? Why?
        2. Does it effectively use the 3 user inputs?
        3. Is the speaking time appropriate (1-1.5 minutes)?
        4. Are there enough complex sentences and varied structures?
        5. What could be improved?

        Output format:
        [ANALYSIS]
        (Your analysis)

        [SUGGESTIONS FOR IMPROVEMENT]
        (2-3 concrete suggestions)
        """
        
        # llama_llm을 사용하여 분석 실행
        analysis = llama_llm.predict(prompt)
        
        return analysis

# 스크립트 개선 도구
class ScriptRefiner(BaseTool):
    name: str = "script_refiner"
    description: str = "분석 결과를 바탕으로 OPIC 스크립트를 개선합니다."
    args_schema: type[BaseModel] = ScriptRefinerInput
    
    def _run(self, original_script: str, analysis: str) -> str:
        # 프롬프트 구성
        prompt = f"""
        Refine the following OPIC script based on the analysis:

        [ORIGINAL SCRIPT]
        {original_script}

        [ANALYSIS]
        {analysis}

        Please create an improved version that:
        1. Addresses the issues mentioned in the analysis
        2. Maintains IH level appropriateness
        3. Keeps within 1-1.5 minute speaking time
        4. Preserves the original content but enhances its quality

        Output only the refined script with the same format as the original.
        """
        
        # llama_llm을 사용하여 스크립트 개선
        refined_script = llama_llm.predict(prompt)
        
        return refined_script

# Pydantic 모델을 사용하지 않는 간소화된 버전의 함수로 대체
def generate_script(user_inputs):
    """사용자 입력을 바탕으로 OPIC 스크립트 생성"""
    prompt = f"""
    Generate an OPIC IH (Intermediate High) level script based on the following user inputs:

    [User's Image Description]: {user_inputs['image_description']}
    [Highlight Point]: {user_inputs['highlight_point']}
    [One-word Expression and Reason]: {user_inputs['one_word_expression']}

    Requirements:
    1. Create 6-7 sentences to fit a 1-1.5 minute speaking time
    2. Use complex sentences rather than simple ones
    3. Include appropriate transition phrases and idiomatic expressions for IH level
    4. Create a cohesive flow using all three inputs
    5. Format the output as a first-person narrative
    6. Include relevant details from the user's inputs

    Output format:
    [SCRIPT]
    (Script content)

    [WORD COUNT]: (word count)
    [ESTIMATED SPEAKING TIME]: (estimated time)
    [COMPLEX SENTENCES]: (number of complex sentences)
    [KEY EXPRESSIONS]: (3-4 key expressions used)
    """
    
    script = llama_llm.predict(prompt)
    return script

def analyze_script(script):
    """생성된 스크립트 분석"""
    prompt = f"""
    Analyze the following OPIC script for IH level and provide improvement suggestions:

    {script}

    Please analyze:
    1. Is it appropriate for IH level? Why?
    2. Is the speaking time appropriate (1-1.5 minutes)?
    3. Are there enough complex sentences and varied structures?
    4. What could be improved?

    Output format:
    [ANALYSIS]
    (Your analysis)

    [SUGGESTIONS FOR IMPROVEMENT]
    (2-3 concrete suggestions)
    """
    
    analysis = llama_llm.predict(prompt)
    return analysis

def refine_script(original_script, analysis):
    """분석 결과를 바탕으로 스크립트 개선"""
    prompt = f"""
    Refine the following OPIC script based on the analysis:

    [ORIGINAL SCRIPT]
    {original_script}

    [ANALYSIS]
    {analysis}

    Please create an improved version that:
    1. Addresses the issues mentioned in the analysis
    2. Maintains IH level appropriateness
    3. Keeps within 1-1.5 minute speaking time
    4. Preserves the original content but enhances its quality

    Output only the refined script with the same format as the original.
    """
    
    refined_script = llama_llm.predict(prompt)
    return refined_script

def create_opic_react_agent():
    # 도구 초기화
    tools = [
        ScriptGenerator(),
        ScriptAnalyzer(),
        ScriptRefiner()
    ]
    
    # 시스템 메시지 정의
    system_message = SystemMessage(
        content="""You are an expert OPIC script generator assistant.

Your goal is to create high-quality OPIC IH (Intermediate High) level scripts based on user inputs about a description.

The user will provide answers to these three questions:
1. "지금 머릿속에 가장 먼저 떠오르는 이미지는 어떤 것인가요? 그 장면을 조금만 더 자세히 묘사해 주시겠어요?" (What image comes to mind first? Can you describe that scene in more detail?)
2. "만약 친구나 가족에게 이 장소(또는 대상)를 소개한다면 어떤 부분을 가장 강조하고 싶은가요?" (If you were to introduce this place or object to friends or family, what aspects would you emphasize?)
3. "그 대상이나 장소를 한 마디로 표현한다면 무엇이라고 표현할 수 있을까요? 이유도 알려주세요." (If you had to describe this place or object in one word, what would it be and why?)

You have access to these tools:
- script_generator: Creates an initial OPIC script based on user inputs
- script_analyzer: Analyzes the script for quality and fit to IH level
- script_refiner: Refines the script based on analysis

Follow this process:
1. Understand the user's inputs about their description
2. Determine the topic of the description (restaurant, travel, hobby, etc.)
3. Use script_generator to create an initial script
4. Use script_analyzer to evaluate the script
5. Use script_refiner to improve the script if needed
6. Present the final script to the user

Always think step by step and reason through your decisions.
"""
    )
    
    # 메모리 초기화
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    
    # 에이전트 초기화
    agent = initialize_agent(
        tools,
        llama_llm,
        agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        memory=memory,
        system_message=system_message,
        extra_prompt_messages=[MessagesPlaceholder(variable_name="chat_history")]
    )
    
    return agent

def extract_user_inputs(user_text):
    """사용자 입력에서 세 가지 질문에 대한 답변을 추출"""
    
    # 정규 표현식을 사용하여 패턴 매칭
    # 실제 입력 형식에 따라 조정 필요
    inputs = {
        "image_description": "",
        "highlight_point": "",
        "one_word_expression": ""
    }
    
    # 입력 텍스트를 줄 단위로 분리
    lines = user_text.strip().split('\n')
    
    # 각 답변을 추출
    current_question = 0
    buffer = []
    
    for line in lines:
        if "머릿속에 가장 먼저 떠오르는 이미지" in line or "그 장면을 조금만 더 자세히 묘사" in line:
            current_question = 1
            continue
        elif "친구나 가족에게 이 장소" in line or "어떤 부분을 가장 강조" in line:
            if current_question == 1 and buffer:
                inputs["image_description"] = ' '.join(buffer)
                buffer = []
            current_question = 2
            continue
        elif "한 마디로 표현" in line or "이유도 알려주세요" in line:
            if current_question == 2 and buffer:
                inputs["highlight_point"] = ' '.join(buffer)
                buffer = []
            current_question = 3
            continue
        elif line.strip():  # 비어있지 않은 줄만 처리
            buffer.append(line.strip())
    
    # 마지막 질문 처리
    if current_question == 3 and buffer:
        inputs["one_word_expression"] = ' '.join(buffer)
    
    # 간단한 휴리스틱: 질문이 명시적으로 없는 경우
    if not inputs["image_description"] and len(lines) >= 3:
        # 첫 번째 답변은 첫 번째 줄
        inputs["image_description"] = lines[0].strip()
        # 두 번째 답변은 두 번째 줄
        if len(lines) > 1:
            inputs["highlight_point"] = lines[1].strip()
        # 세 번째 답변은 세 번째 줄
        if len(lines) > 2:
            inputs["one_word_expression"] = lines[2].strip()
    
    return inputs

def generate_opic_description_script(user_text):
    """
    사용자 입력을 바탕으로 OPIC 묘사 스크립트를 생성
    
    Args:
        user_text (str): 사용자가 입력한 세 가지 질문에 대한 답변
        
    Returns:
        str: 생성된 OPIC 스크립트
    """
    # 사용자 입력 추출
    inputs = extract_user_inputs(user_text)
    
    # 입력이 비어있는지 확인
    if not inputs["image_description"] or not inputs["highlight_point"] or not inputs["one_word_expression"]:
        return "입력이 불완전합니다. 세 가지 질문에 대한 답변을 모두 제공해주세요."
    
    try:
        # ReAct 에이전트 사용 시도
        agent = create_opic_react_agent()
        
        prompt = f"""
        다음은 OPIC 묘사 질문에 대한 사용자의 답변입니다:
        
        1. 떠오르는 이미지/장면 묘사: {inputs['image_description']}
        2. 강조하고 싶은 부분: {inputs['highlight_point']}
        3. 한 마디로 표현 및 이유: {inputs['one_word_expression']}
        
        이 정보를 바탕으로 OPIC IH 수준의 묘사 스크립트를 생성해주세요.
        """
        
        result = agent.run(prompt)
        return result
    
    except Exception as e:
        print(f"에이전트 실행 중 오류 발생: {e}")
        print("대체 방법으로 스크립트 생성을 시도합니다...")
        
        # 대체 방법: 단계적으로 함수 호출
        try:
            # 1단계: 초기 스크립트 생성
            initial_script = generate_script(inputs)
            
            # 2단계: 스크립트 분석
            script_analysis = analyze_script(initial_script)
            
            # 3단계: 스크립트 개선
            final_script = refine_script(initial_script, script_analysis)
            
            return final_script
        
        except Exception as inner_e:
            print(f"대체 방법 실행 중 오류 발생: {inner_e}")
            
            # 최종 대안: 직접 스크립트 생성
            prompt = f"""
            Generate an OPIC IH (Intermediate High) level script based on these inputs:
            
            [Image Description]: {inputs['image_description']}
            [Highlight Point]: {inputs['highlight_point']}
            [One-word Expression]: {inputs['one_word_expression']}
            
            Create a 1-1.5 minute script with 6-7 complex sentences using appropriate 
            idioms and transitions for IH level. Include all the user's key points.
            """
            
            return llama_llm.predict(prompt)

# 사용 예시
if __name__ == "__main__":
    # 샘플 사용자 입력
    sample_user_input = """
    지금 머릿속에 가장 먼저 떠오르는 이미지는 어떤 것인가요? 그 장면을 조금만 더 자세히 묘사해 주시겠어요?
    바다가 보이는 작은 카페에 앉아 있는 모습이 떠올라요. 하얀 테이블 위에 아이스 아메리카노가 놓여있고, 창문 너머로 푸른 바다와 하늘이 맞닿아 있어요. 바닷가에는 몇몇 사람들이 산책하고 있고, 파도 소리가 은은하게 들려요.
    
    만약 친구나 가족에게 이 장소(또는 대상)를 소개한다면 어떤 부분을 가장 강조하고 싶은가요?
    바다를 바라보며 여유롭게 커피를 마실 수 있는 분위기를 강조하고 싶어요. 특히 해질녘에는 노을이 바다에 반사되어 정말 아름다운 풍경을 볼 수 있어요. 또한 카페의 디저트도 맛있고, 사장님이 친절하셔서 편안한 시간을 보낼 수 있어요.
    
    그 대상이나 장소를 한 마디로 표현한다면 무엇이라고 표현할 수 있을까요? 이유도 알려주세요.
    '힐링'이라고 표현할 수 있을 것 같아요. 일상에서 벗어나 잠시 바다를 바라보며 커피 한 잔 마시는 시간이 마음의 평화를 가져다 주고 재충전할 수 있게 해주기 때문이에요.
    """
    
    print("사용자 입력:")
    print(sample_user_input)
    print("\n" + "="*50 + "\n")
    
    script = generate_opic_description_script(sample_user_input)
    print("생성된 OPIC 스크립트:")
    print(script)

사용자 입력:

    지금 머릿속에 가장 먼저 떠오르는 이미지는 어떤 것인가요? 그 장면을 조금만 더 자세히 묘사해 주시겠어요?
    바다가 보이는 작은 카페에 앉아 있는 모습이 떠올라요. 하얀 테이블 위에 아이스 아메리카노가 놓여있고, 창문 너머로 푸른 바다와 하늘이 맞닿아 있어요. 바닷가에는 몇몇 사람들이 산책하고 있고, 파도 소리가 은은하게 들려요.
    
    만약 친구나 가족에게 이 장소(또는 대상)를 소개한다면 어떤 부분을 가장 강조하고 싶은가요?
    바다를 바라보며 여유롭게 커피를 마실 수 있는 분위기를 강조하고 싶어요. 특히 해질녘에는 노을이 바다에 반사되어 정말 아름다운 풍경을 볼 수 있어요. 또한 카페의 디저트도 맛있고, 사장님이 친절하셔서 편안한 시간을 보낼 수 있어요.
    
    그 대상이나 장소를 한 마디로 표현한다면 무엇이라고 표현할 수 있을까요? 이유도 알려주세요.
    '힐링'이라고 표현할 수 있을 것 같아요. 일상에서 벗어나 잠시 바다를 바라보며 커피 한 잔 마시는 시간이 마음의 평화를 가져다 주고 재충전할 수 있게 해주기 때문이에요.
    




> Entering new AgentExecutor chain...
Thought: 사용자의 입력을 바탕으로 OPIC IH 수준의 스크립트를 생성해야 합니다. 사용자가 제공한 정보는 다음과 같습니다: 
1. 떠오르는 이미지/장면 묘사: 바다가 보이는 작은 카페에 앉아 있는 모습
2. 강조하고 싶은 부분: 바다를 바라보며 여유롭게 커피를 마실 수 있는 분위기와 아름다운 풍경
3. 한 마디로 표현 및 이유: '힐링'이라고 표현하며, 이는 일상에서 벗어나 마음의 평화를 찾을 수 있는 장소임을 의미합니다.
이 정보를 바탕으로 스크립트를 생성할 수 있습니다.

Action:
```
{
  "action": "script_generator",
  "action_input": {
    "image_description": "바다가 보이는 작은 카페에 앉아 있는 모습",
   

In [25]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# 사용자 입력에서 답변 추출하는 함수
def extract_user_inputs(user_text):
    """사용자 입력에서 세 가지 질문에 대한 답변을 추출"""
    
    inputs = {
        "image_description": "",
        "highlight_point": "",
        "one_word_expression": ""
    }
    
    # 입력 텍스트를 줄 단위로 분리
    lines = user_text.strip().split('\n')
    
    # 각 답변을 추출
    current_question = 0
    buffer = []
    
    for line in lines:
        if "머릿속에 가장 먼저 떠오르는 이미지" in line or "그 장면을 조금만 더 자세히 묘사" in line:
            current_question = 1
            continue
        elif "친구나 가족에게 이 장소" in line or "어떤 부분을 가장 강조" in line:
            if current_question == 1 and buffer:
                inputs["image_description"] = ' '.join(buffer)
                buffer = []
            current_question = 2
            continue
        elif "한 마디로 표현" in line or "이유도 알려주세요" in line:
            if current_question == 2 and buffer:
                inputs["highlight_point"] = ' '.join(buffer)
                buffer = []
            current_question = 3
            continue
        elif line.strip():  # 비어있지 않은 줄만 처리
            buffer.append(line.strip())
    
    # 마지막 질문 처리
    if current_question == 3 and buffer:
        inputs["one_word_expression"] = ' '.join(buffer)
    
    # 질문이 명시적으로 없는 경우 간단한 처리
    if not inputs["image_description"] and len(lines) >= 3:
        sections = []
        current_section = []
        
        for line in lines:
            if not line.strip():
                if current_section:
                    sections.append(' '.join(current_section))
                    current_section = []
            else:
                current_section.append(line.strip())
        
        if current_section:
            sections.append(' '.join(current_section))
        
        if len(sections) >= 1:
            inputs["image_description"] = sections[0]
        if len(sections) >= 2:
            inputs["highlight_point"] = sections[1]
        if len(sections) >= 3:
            inputs["one_word_expression"] = sections[2]
    
    return inputs

def generate_opic_script_english_only(user_text):
    """
    순수 영어로만 OPIC 묘사 스크립트를 생성하는 함수
    
    Args:
        user_text (str): 사용자가 입력한 세 가지 질문에 대한 답변
        
    Returns:
        str: 생성된 OPIC 영어 스크립트
    """
    # 사용자 입력 추출
    inputs = extract_user_inputs(user_text)
    
    # 입력이 비어있는지 확인
    if not inputs["image_description"] or not inputs["highlight_point"] or not inputs["one_word_expression"]:
        return "입력이 불완전합니다. 세 가지 질문에 대한 답변을 모두 제공해주세요."
    
    # 프롬프트 템플릿 생성
    template = """
    You are an expert OPIC script generator for English learners preparing for the OPIC test.
    
    Create a script based on the following three user responses about a description:
    
    [Image/Scene Description]:
    {image_description}
    
    [Highlighted Aspects]:
    {highlight_point}
    
    [One-word Expression and Reason]:
    {one_word_expression}
    
    === SCRIPT REQUIREMENTS ===
    1. Create 6-7 sentences to fit a 1-1.5 minute speaking time
    2. Use complex sentences rather than simple ones
    3. Include appropriate transition phrases and idiomatic expressions for IH level
    4. Naturally integrate all three user inputs
    5. Format as a first-person narrative with personal feelings
    6. Maintain clear structure and natural flow
    7. USE ONLY STANDARD ENGLISH - no foreign words or expressions
    8. Include only natural English idioms and transitions like "personally," "in addition," "furthermore," etc.
    
    Output format:
    [SCRIPT]
    (Script content - ENGLISH ONLY)
    
    [WORD COUNT]: (word count)
    [ESTIMATED SPEAKING TIME]: (estimated time)
    [COMPLEX SENTENCES]: (number of complex sentences)
    [KEY EXPRESSIONS]: (3-4 English idioms or expressions used)
    """
    
    # 프롬프트 객체 생성
    prompt = PromptTemplate(
        template=template,
        input_variables=["image_description", "highlight_point", "one_word_expression"]
    )
    
    # 체인 생성 및 실행
    chain = LLMChain(llm=llama_llm, prompt=prompt)
    script = chain.run(inputs)
    
    return script

# 사용 예시
if __name__ == "__main__":
    # 샘플 사용자 입력
    sample_user_input = """
    지금 머릿속에 가장 먼저 떠오르는 이미지는 어떤 것인가요? 그 장면을 조금만 더 자세히 묘사해 주시겠어요?
    바다가 보이는 작은 카페에 앉아 있는 모습이 떠올라요. 하얀 테이블 위에 아이스 아메리카노가 놓여있고, 창문 너머로 푸른 바다와 하늘이 맞닿아 있어요. 바닷가에는 몇몇 사람들이 산책하고 있고, 파도 소리가 은은하게 들려요.
    
    만약 친구나 가족에게 이 장소(또는 대상)를 소개한다면 어떤 부분을 가장 강조하고 싶은가요?
    바다를 바라보며 여유롭게 커피를 마실 수 있는 분위기를 강조하고 싶어요. 특히 해질녘에는 노을이 바다에 반사되어 정말 아름다운 풍경을 볼 수 있어요. 또한 카페의 디저트도 맛있고, 사장님이 친절하셔서 편안한 시간을 보낼 수 있어요.
    
    그 대상이나 장소를 한 마디로 표현한다면 무엇이라고 표현할 수 있을까요? 이유도 알려주세요.
    '힐링'이라고 표현할 수 있을 것 같아요. 일상에서 벗어나 잠시 바다를 바라보며 커피 한 잔 마시는 시간이 마음의 평화를 가져다 주고 재충전할 수 있게 해주기 때문이에요.
    """
    
    print("사용자 입력:")
    print(sample_user_input)
    print("\n" + "="*50 + "\n")
    
    script = generate_opic_script_english_only(sample_user_input)
    print("생성된 OPIC 스크립트:")
    print(script)

사용자 입력:

    지금 머릿속에 가장 먼저 떠오르는 이미지는 어떤 것인가요? 그 장면을 조금만 더 자세히 묘사해 주시겠어요?
    바다가 보이는 작은 카페에 앉아 있는 모습이 떠올라요. 하얀 테이블 위에 아이스 아메리카노가 놓여있고, 창문 너머로 푸른 바다와 하늘이 맞닿아 있어요. 바닷가에는 몇몇 사람들이 산책하고 있고, 파도 소리가 은은하게 들려요.
    
    만약 친구나 가족에게 이 장소(또는 대상)를 소개한다면 어떤 부분을 가장 강조하고 싶은가요?
    바다를 바라보며 여유롭게 커피를 마실 수 있는 분위기를 강조하고 싶어요. 특히 해질녘에는 노을이 바다에 반사되어 정말 아름다운 풍경을 볼 수 있어요. 또한 카페의 디저트도 맛있고, 사장님이 친절하셔서 편안한 시간을 보낼 수 있어요.
    
    그 대상이나 장소를 한 마디로 표현한다면 무엇이라고 표현할 수 있을까요? 이유도 알려주세요.
    '힐링'이라고 표현할 수 있을 것 같아요. 일상에서 벗어나 잠시 바다를 바라보며 커피 한 잔 마시는 시간이 마음의 평화를 가져다 주고 재충전할 수 있게 해주기 때문이에요.
    


생성된 OPIC 스크립트:
[SCRIPT]
As I sit in a small cafe overlooking the ocean, I am reminded of the serenity that this scene brings me. Personally, I find that gazing out at the sea while sipping on a cup of iced coffee creates a sense of tranquility that is hard to find in our busy lives. Furthermore, the atmosphere is particularly breathtaking during sunset, when the sky is painted with hues of orange and pink, and the waves gently lap 

In [27]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import re

def extract_user_inputs_fixed(user_text):
    """
    개선된 사용자 입력 추출 함수 - 정규표현식 사용
    """
    # 정규표현식을 사용하여 각 Q&A 쌍 추출
    q1_pattern = r"Q: 지금 머릿속에 가장 먼저 떠오르는 장면.+?👉\s*(.+?)(?=Q:|$)"
    q2_pattern = r"Q: 친구나 가족에게 이 장소를 소개.+?👉\s*(.+?)(?=Q:|$)"
    q3_pattern = r"Q: 이 장소를 한마디로 표현.+?👉\s*(.+?)(?=Q:|$)"
    
    # 정규표현식 매칭
    image_match = re.search(q1_pattern, user_text, re.DOTALL)
    highlight_match = re.search(q2_pattern, user_text, re.DOTALL)
    expression_match = re.search(q3_pattern, user_text, re.DOTALL)
    
    # 결과 추출
    inputs = {
        "image_description": image_match.group(1).strip() if image_match else "",
        "highlight_point": highlight_match.group(1).strip() if highlight_match else "",
        "one_word_expression": expression_match.group(1).strip() if expression_match else ""
    }
    
    return inputs

def generate_opic_script_english_only(user_text):
    """
    순수 영어로만 OPIC 묘사 스크립트를 생성하는 함수
    
    Args:
        user_text (str): 사용자가 입력한 세 가지 질문에 대한 답변
        
    Returns:
        str: 생성된 OPIC 스크립트
    """
    # 사용자 입력 추출 - 수정된 함수 사용
    inputs = extract_user_inputs_fixed(user_text)
    
    # 입력이 비어있는지 확인
    if not inputs["image_description"] or not inputs["highlight_point"] or not inputs["one_word_expression"]:
        return f"""입력이 불완전합니다. 세 가지 질문에 대한 답변을 모두 제공해주세요.
        
추출된 값:
이미지 묘사: {inputs["image_description"]}
강조점: {inputs["highlight_point"]}
한마디 표현: {inputs["one_word_expression"]}"""
    
    # 프롬프트 템플릿 생성
    template = """
    You are an expert OPIC script generator for English learners preparing for the OPIC test.
    
    Create a script based on the following three user responses about a description:
    
    [Image/Scene Description]:
    {image_description}
    
    [Highlighted Aspects]:
    {highlight_point}
    
    [One-word Expression and Reason]:
    {one_word_expression}
    
    === SCRIPT REQUIREMENTS ===
    1. Create 6-7 sentences to fit a 1-1.5 minute speaking time
    2. Use complex sentences rather than simple ones
    3. Include appropriate transition phrases and idiomatic expressions for IH level
    4. Naturally integrate all three user inputs
    5. Format as a first-person narrative with personal feelings
    6. Maintain clear structure and natural flow
    7. USE ONLY STANDARD ENGLISH - no foreign words or expressions
    8. Include only natural English idioms and transitions like "personally," "in addition," "furthermore," etc.
    
    Output format:
    [SCRIPT]
    (Script content - ENGLISH ONLY)
    
    [WORD COUNT]: (word count)
    [ESTIMATED SPEAKING TIME]: (estimated time)
    [COMPLEX SENTENCES]: (number of complex sentences)
    [KEY EXPRESSIONS]: (3-4 English idioms or expressions used)
    """
    
    # 프롬프트 객체 생성
    prompt = PromptTemplate(
        template=template,
        input_variables=["image_description", "highlight_point", "one_word_expression"]
    )
    
    # 체인 생성 및 실행
    chain = LLMChain(llm=llama_llm, prompt=prompt)
    script = chain.run(inputs)
    
    return script

# 직접 테스트용 함수
def test_extraction():
    """정규식 기반 추출 테스트 함수"""
    test_input = """
    Q: 지금 머릿속에 가장 먼저 떠오르는 장면을 말해 줄래? 👉 바다가 보이는 작은 카페가 떠올라요. 하얀 테이블 위에 아이스 아메리카노가 있고, 창문 밖에는 파란 바다가 보여요. 사람들이 바닷가를 걸어 다니고, 파도 소리가 들려요.
    Q: 친구나 가족에게 이 장소를 소개한다면 어떤 점을 말하고 싶어? 👉 바다를 보면서 커피를 마실 수 있어서 좋아요. 해가 질 때 하늘이 예쁘고, 바다도 반짝거려요. 카페에 맛있는 디저트도 있고, 사장님이 친절해요!
    Q: 이 장소를 한마디로 표현하면? 👉 "편안한 곳!" 왜냐하면 여기 오면 기분이 좋아지고, 푹 쉴 수 있어요!
    """
    
    result = extract_user_inputs_fixed(test_input)
    return result

# 명시적인 예제 프롬프트를 만들어 테스트하는 함수
def test_with_direct_llm_call():
    """LLM에 직접 프롬프트를 전달하여 테스트"""
    test_input = """
    Q: 지금 머릿속에 가장 먼저 떠오르는 장면을 말해 줄래? 👉 바다가 보이는 작은 카페가 떠올라요. 하얀 테이블 위에 아이스 아메리카노가 있고, 창문 밖에는 파란 바다가 보여요. 사람들이 바닷가를 걸어 다니고, 파도 소리가 들려요.
    Q: 친구나 가족에게 이 장소를 소개한다면 어떤 점을 말하고 싶어? 👉 바다를 보면서 커피를 마실 수 있어서 좋아요. 해가 질 때 하늘이 예쁘고, 바다도 반짝거려요. 카페에 맛있는 디저트도 있고, 사장님이 친절해요!
    Q: 이 장소를 한마디로 표현하면? 👉 "편안한 곳!" 왜냐하면 여기 오면 기분이 좋아지고, 푹 쉴 수 있어요!
    """
    
    # 먼저 입력을 추출
    inputs = extract_user_inputs_fixed(test_input)
    
    # 명시적인 프롬프트 구성
    direct_prompt = f"""
    You are an expert OPIC script generator for English learners preparing for the OPIC test.
    
    Create a script based on the following three user responses about a description:
    
    [Image/Scene Description]:
    {inputs["image_description"]}
    
    [Highlighted Aspects]:
    {inputs["highlight_point"]}
    
    [One-word Expression and Reason]:
    {inputs["one_word_expression"]}
    
    === SCRIPT REQUIREMENTS ===
    1. Create 6-7 sentences to fit a 1-1.5 minute speaking time
    2. Use complex sentences rather than simple ones
    3. Include appropriate transition phrases and idiomatic expressions for IH level
    4. Naturally integrate all three user inputs
    5. Format as a first-person narrative with personal feelings
    6. Maintain clear structure and natural flow
    7. USE ONLY STANDARD ENGLISH - no foreign words or expressions
    8. Include only natural English idioms and transitions like "personally," "in addition," "furthermore," etc.
    
    Output format:
    [SCRIPT]
    (Script content - ENGLISH ONLY)
    
    [WORD COUNT]: (word count)
    [ESTIMATED SPEAKING TIME]: (estimated time)
    [COMPLEX SENTENCES]: (number of complex sentences)
    [KEY EXPRESSIONS]: (3-4 English idioms or expressions used)
    """
    
    # LLM에 직접 전달
    result = llama_llm.predict(direct_prompt)
    return result

# 사용 예시
if __name__ == "__main__":
    # 샘플 사용자 입력
    sample_user_input = """
    Q: 지금 머릿속에 가장 먼저 떠오르는 장면을 말해 줄래? 👉 바다가 보이는 작은 카페가 떠올라요. 하얀 테이블 위에 아이스 아메리카노가 있고, 창문 밖에는 파란 바다가 보여요. 사람들이 바닷가를 걸어 다니고, 파도 소리가 들려요.
    Q: 친구나 가족에게 이 장소를 소개한다면 어떤 점을 말하고 싶어? 👉 바다를 보면서 커피를 마실 수 있어서 좋아요. 해가 질 때 하늘이 예쁘고, 바다도 반짝거려요. 카페에 맛있는 디저트도 있고, 사장님이 친절해요!
    Q: 이 장소를 한마디로 표현하면? 👉 "편안한 곳!" 왜냐하면 여기 오면 기분이 좋아지고, 푹 쉴 수 있어요!
    """
    
    print("사용자 입력:")
    print(sample_user_input)
    print("\n" + "="*50 + "\n")
    
    # 추출 테스트
    extracted = test_extraction()
    print("추출 테스트 결과:")
    for key, value in extracted.items():
        print(f"{key}: {value}")
    
    print("\n" + "="*50 + "\n")
    
    # 정규식으로 추출한 결과로 스크립트 생성
    print("생성된 OPIC 스크립트:")
    script = generate_opic_script_english_only(sample_user_input)
    print(script)
    
    print("\n" + "="*50 + "\n")
    
    # 직접 LLM 호출 테스트 (대안)
    print("직접 LLM 호출 결과:")
    direct_result = test_with_direct_llm_call()
    print(direct_result)

사용자 입력:

    Q: 지금 머릿속에 가장 먼저 떠오르는 장면을 말해 줄래? 👉 바다가 보이는 작은 카페가 떠올라요. 하얀 테이블 위에 아이스 아메리카노가 있고, 창문 밖에는 파란 바다가 보여요. 사람들이 바닷가를 걸어 다니고, 파도 소리가 들려요.
    Q: 친구나 가족에게 이 장소를 소개한다면 어떤 점을 말하고 싶어? 👉 바다를 보면서 커피를 마실 수 있어서 좋아요. 해가 질 때 하늘이 예쁘고, 바다도 반짝거려요. 카페에 맛있는 디저트도 있고, 사장님이 친절해요!
    Q: 이 장소를 한마디로 표현하면? 👉 "편안한 곳!" 왜냐하면 여기 오면 기분이 좋아지고, 푹 쉴 수 있어요!
    


추출 테스트 결과:
image_description: 바다가 보이는 작은 카페가 떠올라요. 하얀 테이블 위에 아이스 아메리카노가 있고, 창문 밖에는 파란 바다가 보여요. 사람들이 바닷가를 걸어 다니고, 파도 소리가 들려요.
highlight_point: 바다를 보면서 커피를 마실 수 있어서 좋아요. 해가 질 때 하늘이 예쁘고, 바다도 반짝거려요. 카페에 맛있는 디저트도 있고, 사장님이 친절해요!
one_word_expression: "편안한 곳!" 왜냐하면 여기 오면 기분이 좋아지고, 푹 쉴 수 있어요!


생성된 OPIC 스크립트:
[SCRIPT]
As I sit in this quaint little cafe, I am reminded of a place that holds a special spot in my heart - a small coffee shop with a stunning view of the ocean. Personally, I find it incredibly appealing to be able to enjoy a cup of coffee while taking in the breathtaking scenery, and I must say, it's an experience like no other. Furthermore, the at

In [30]:
from langchain.agents import AgentType, initialize_agent
from langchain.tools import BaseTool
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.schema import SystemMessage
from typing import Dict, List, Optional, Any
from pydantic import BaseModel, Field
import re
from langchain_groq import ChatGroq


# 도구 입력 모델 정의
class ScriptGeneratorInput(BaseModel):
    image_description: str = Field(..., description="사용자가 떠올린 이미지나 장면에 대한 묘사")
    highlight_point: str = Field(..., description="친구나 가족에게 소개할 때 강조하고 싶은 부분")
    one_word_expression: str = Field(..., description="그 대상이나 장소를 한 마디로 표현한 것과 이유")
    topic: Optional[str] = Field(None, description="OPIC 질문의 주제 (예: 식당, 여행, 취미 등)")

class ScriptAnalyzerInput(BaseModel):
    script: str = Field(..., description="분석할 OPIC 스크립트")

class ScriptRefinerInput(BaseModel):
    original_script: str = Field(..., description="원본 OPIC 스크립트")
    analysis: str = Field(..., description="스크립트 분석 결과")

# 스크립트 생성 도구
class ScriptGenerator(BaseTool):
    name: str = "script_generator"
    description: str = "사용자 입력을 바탕으로 OPIC IH 수준의 스크립트를 생성합니다."
    args_schema: type[BaseModel] = ScriptGeneratorInput
    
    def _run(self, image_description: str, highlight_point: str, one_word_expression: str, topic: Optional[str] = None) -> str:
        # 프롬프트 구성
        prompt = f"""
        Generate an OPIC IH (Intermediate High) level script based on the following user inputs:

        [User's Image Description]: {image_description}
        [Highlight Point]: {highlight_point}
        [One-word Expression and Reason]: {one_word_expression}
        {f"[Topic]: {topic}" if topic else ""}

        Requirements:
        1. Create 6-7 sentences to fit a 1-1.5 minute speaking time
        2. Use complex sentences rather than simple ones
        3. Include appropriate transition phrases and idiomatic expressions for IH level
        4. Create a cohesive flow using all three inputs
        5. Format the output as a first-person narrative
        6. Include relevant details from the user's inputs

        Output format:
        [SCRIPT]
        (Script content)

        [WORD COUNT]: (word count)
        [ESTIMATED SPEAKING TIME]: (estimated time)
        [COMPLEX SENTENCES]: (number of complex sentences)
        [KEY EXPRESSIONS]: (3-4 key expressions used)
        """
        
        # llama_llm을 사용하여 스크립트 생성
        script = llama_llm.predict(prompt)
        
        return script

# 스크립트 분석 도구
class ScriptAnalyzer(BaseTool):
    name: str = "script_analyzer"
    description: str = "생성된 OPIC 스크립트를 분석하여 개선점을 제시합니다."
    args_schema: type[BaseModel] = ScriptAnalyzerInput
    
    def _run(self, script: str) -> str:
        # 프롬프트 구성
        prompt = f"""
        Analyze the following OPIC script for IH level and provide improvement suggestions:

        {script}

        Please analyze:
        1. Is it appropriate for IH level? Why?
        2. Does it effectively use the 3 user inputs?
        3. Is the speaking time appropriate (1-1.5 minutes)?
        4. Are there enough complex sentences and varied structures?
        5. What could be improved?

        Output format:
        [ANALYSIS]
        (Your analysis)

        [SUGGESTIONS FOR IMPROVEMENT]
        (2-3 concrete suggestions)
        """
        
        # llama_llm을 사용하여 분석 실행
        analysis = llama_llm.predict(prompt)
        
        return analysis

# 스크립트 개선 도구
class ScriptRefiner(BaseTool):
    name: str = "script_refiner"
    description: str = "분석 결과를 바탕으로 OPIC 스크립트를 개선합니다."
    args_schema: type[BaseModel] = ScriptRefinerInput
    
    def _run(self, original_script: str, analysis: str) -> str:
        # 프롬프트 구성
        prompt = f"""
        Refine the following OPIC script based on the analysis:

        [ORIGINAL SCRIPT]
        {original_script}

        [ANALYSIS]
        {analysis}

        Please create an improved version that:
        1. Addresses the issues mentioned in the analysis
        2. Maintains IH level appropriateness
        3. Keeps within 1-1.5 minute speaking time
        4. Preserves the original content but enhances its quality

        Output only the refined script with the same format as the original.
        """
        
        # llama_llm을 사용하여 스크립트 개선
        refined_script = llama_llm.predict(prompt)
        
        return refined_script

# 사용자 입력 추출 함수
def extract_user_inputs(user_text):
    """사용자 입력에서 세 가지 질문에 대한 답변을 추출"""
    
    # 정규 표현식을 사용하여 패턴 매칭
    inputs = {
        "image_description": "",
        "highlight_point": "",
        "one_word_expression": ""
    }
    
    # 입력 텍스트를 줄 단위로 분리
    lines = user_text.strip().split('\n')
    
    # 각 답변을 추출
    current_question = 0
    buffer = []
    
    for line in lines:
        if "머릿속에 가장 먼저 떠오르는 이미지" in line or "그 장면을 조금만 더 자세히 묘사" in line:
            current_question = 1
            continue
        elif "친구나 가족에게 이 장소" in line or "어떤 부분을 가장 강조" in line:
            if current_question == 1 and buffer:
                inputs["image_description"] = ' '.join(buffer)
                buffer = []
            current_question = 2
            continue
        elif "한 마디로 표현" in line or "이유도 알려주세요" in line:
            if current_question == 2 and buffer:
                inputs["highlight_point"] = ' '.join(buffer)
                buffer = []
            current_question = 3
            continue
        elif line.strip():  # 비어있지 않은 줄만 처리
            buffer.append(line.strip())
    
    # 마지막 질문 처리
    if current_question == 3 and buffer:
        inputs["one_word_expression"] = ' '.join(buffer)
    
    # 간단한 휴리스틱: 질문이 명시적으로 없는 경우
    if not inputs["image_description"] and len(lines) >= 3:
        # 첫 번째 답변은 첫 번째 줄
        inputs["image_description"] = lines[0].strip()
        # 두 번째 답변은 두 번째 줄
        if len(lines) > 1:
            inputs["highlight_point"] = lines[1].strip()
        # 세 번째 답변은 세 번째 줄
        if len(lines) > 2:
            inputs["one_word_expression"] = lines[2].strip()
    
    return inputs

# ReAct 에이전트 생성 함수
def create_opic_react_agent():
    # 도구 초기화
    tools = [
        ScriptGenerator(),
        ScriptAnalyzer(),
        ScriptRefiner()
    ]
    
    # 시스템 메시지 정의
    system_message = SystemMessage(
        content="""You are an expert OPIC script generator assistant.

Your goal is to create high-quality OPIC IH (Intermediate High) level scripts based on user inputs about a description.

The user will provide answers to these three questions:
1. "지금 머릿속에 가장 먼저 떠오르는 이미지는 어떤 것인가요? 그 장면을 조금만 더 자세히 묘사해 주시겠어요?" (What image comes to mind first? Can you describe that scene in more detail?)
2. "만약 친구나 가족에게 이 장소(또는 대상)를 소개한다면 어떤 부분을 가장 강조하고 싶은가요?" (If you were to introduce this place or object to friends or family, what aspects would you emphasize?)
3. "그 대상이나 장소를 한 마디로 표현한다면 무엇이라고 표현할 수 있을까요? 이유도 알려주세요." (If you had to describe this place or object in one word, what would it be and why?)

You have access to these tools:
- script_generator: Creates an initial OPIC script based on user inputs
- script_analyzer: Analyzes the script for quality and fit to IH level
- script_refiner: Refines the script based on analysis

Follow this process:
1. Understand the user's inputs about their description
2. Determine the topic of the description (restaurant, travel, hobby, etc.)
3. Use script_generator to create an initial script
4. Use script_analyzer to evaluate the script
5. Use script_refiner to improve the script if needed
6. Present the final script to the user

Always think step by step and reason through your decisions.
"""
    )
    
    # 메모리 초기화
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    
    # 에이전트 초기화
    agent = initialize_agent(
        tools,
        llama_llm,
        agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        memory=memory,
        system_message=system_message,
        extra_prompt_messages=[MessagesPlaceholder(variable_name="chat_history")]
    )
    
    return agent

# 백업 함수 구현
def generate_script(user_inputs):
    """사용자 입력을 바탕으로 OPIC 스크립트 생성"""
    prompt = f"""
    Generate an OPIC IH (Intermediate High) level script based on the following user inputs:

    [User's Image Description]: {user_inputs['image_description']}
    [Highlight Point]: {user_inputs['highlight_point']}
    [One-word Expression and Reason]: {user_inputs['one_word_expression']}

    Requirements:
    1. Create 6-7 sentences to fit a 1-1.5 minute speaking time
    2. Use complex sentences rather than simple ones
    3. Include appropriate transition phrases and idiomatic expressions for IH level
    4. Create a cohesive flow using all three inputs
    5. Format the output as a first-person narrative
    6. Include relevant details from the user's inputs

    Output format:
    [SCRIPT]
    (Script content)

    [WORD COUNT]: (word count)
    [ESTIMATED SPEAKING TIME]: (estimated time)
    [COMPLEX SENTENCES]: (number of complex sentences)
    [KEY EXPRESSIONS]: (3-4 key expressions used)
    """
    
    script = llama_llm.predict(prompt)
    return script

def analyze_script(script):
    """생성된 스크립트 분석"""
    prompt = f"""
    Analyze the following OPIC script for IH level and provide improvement suggestions:

    {script}

    Please analyze:
    1. Is it appropriate for IH level? Why?
    2. Is the speaking time appropriate (1-1.5 minutes)?
    3. Are there enough complex sentences and varied structures?
    4. What could be improved?

    Output format:
    [ANALYSIS]
    (Your analysis)

    [SUGGESTIONS FOR IMPROVEMENT]
    (2-3 concrete suggestions)
    """
    
    analysis = llama_llm.predict(prompt)
    return analysis

def refine_script(original_script, analysis):
    """분석 결과를 바탕으로 스크립트 개선"""
    prompt = f"""
    Refine the following OPIC script based on the analysis:

    [ORIGINAL SCRIPT]
    {original_script}

    [ANALYSIS]
    {analysis}

    Please create an improved version that:
    1. Addresses the issues mentioned in the analysis
    2. Maintains IH level appropriateness
    3. Keeps within 1-1.5 minute speaking time
    4. Preserves the original content but enhances its quality

    Output only the refined script with the same format as the original.
    """
    
    refined_script = llama_llm.predict(prompt)
    return refined_script

# 메인 함수
def generate_opic_description_script(user_text):
    """
    사용자 입력을 바탕으로 OPIC 묘사 스크립트를 생성
    
    Args:
        user_text (str): 사용자가 입력한 세 가지 질문에 대한 답변
        
    Returns:
        str: 생성된 OPIC 스크립트
    """
    # 사용자 입력 추출
    inputs = extract_user_inputs(user_text)
    
    # 입력이 비어있는지 확인
    if not inputs["image_description"] or not inputs["highlight_point"] or not inputs["one_word_expression"]:
        return "입력이 불완전합니다. 세 가지 질문에 대한 답변을 모두 제공해주세요."
    
    try:
        # ReAct 에이전트 사용 시도
        agent = create_opic_react_agent()
        
        prompt = f"""
        다음은 OPIC 묘사 질문에 대한 사용자의 답변입니다:
        
        1. 떠오르는 이미지/장면 묘사: {inputs['image_description']}
        2. 강조하고 싶은 부분: {inputs['highlight_point']}
        3. 한 마디로 표현 및 이유: {inputs['one_word_expression']}
        
        이 정보를 바탕으로 OPIC IH 수준의 묘사 스크립트를 생성해주세요.
        """
        
        result = agent.run(prompt)
        return result
    
    except Exception as e:
        print(f"에이전트 실행 중 오류 발생: {e}")
        print("대체 방법으로 스크립트 생성을 시도합니다...")
        
        # 대체 방법: 단계적으로 함수 호출
        try:
            # 1단계: 초기 스크립트 생성
            initial_script = generate_script(inputs)
            
            # 2단계: 스크립트 분석
            script_analysis = analyze_script(initial_script)
            
            # 3단계: 스크립트 개선
            final_script = refine_script(initial_script, script_analysis)
            
            return final_script
        
        except Exception as inner_e:
            print(f"대체 방법 실행 중 오류 발생: {inner_e}")
            
            # 최종 대안: 직접 스크립트 생성
            prompt = f"""
            Generate an OPIC IH (Intermediate High) level script based on these inputs:
            
            [Image Description]: {inputs['image_description']}
            [Highlight Point]: {inputs['highlight_point']}
            [One-word Expression]: {inputs['one_word_expression']}
            
            Create a 1-1.5 minute script with 6-7 complex sentences using appropriate 
            idioms and transitions for IH level. Include all the user's key points.
            """
            
            return llama_llm.predict(prompt)

# 사용 예시
if __name__ == "__main__":
    # 샘플 사용자 입력
    sample_user_input = """
    지금 머릿속에 가장 먼저 떠오르는 이미지는 어떤 것인가요? 그 장면을 조금만 더 자세히 묘사해 주시겠어요?
    바다가 보이는 작은 카페에 앉아 있는 모습이 떠올라요. 하얀 테이블 위에 아이스 아메리카노가 놓여있고, 창문 너머로 푸른 바다와 하늘이 맞닿아 있어요. 바닷가에는 몇몇 사람들이 산책하고 있고, 파도 소리가 은은하게 들려요.
    
    만약 친구나 가족에게 이 장소(또는 대상)를 소개한다면 어떤 부분을 가장 강조하고 싶은가요?
    바다를 바라보며 여유롭게 커피를 마실 수 있는 분위기를 강조하고 싶어요. 특히 해질녘에는 노을이 바다에 반사되어 정말 아름다운 풍경을 볼 수 있어요. 또한 카페의 디저트도 맛있고, 사장님이 친절하셔서 편안한 시간을 보낼 수 있어요.
    
    그 대상이나 장소를 한 마디로 표현한다면 무엇이라고 표현할 수 있을까요? 이유도 알려주세요.
    '힐링'이라고 표현할 수 있을 것 같아요. 일상에서 벗어나 잠시 바다를 바라보며 커피 한 잔 마시는 시간이 마음의 평화를 가져다 주고 재충전할 수 있게 해주기 때문이에요.
    """
    
    print("사용자 입력:")
    print(sample_user_input)
    print("\n" + "="*50 + "\n")
    
    script = generate_opic_description_script(sample_user_input)
    print("생성된 OPIC 스크립트:")
    print(script)

사용자 입력:

    지금 머릿속에 가장 먼저 떠오르는 이미지는 어떤 것인가요? 그 장면을 조금만 더 자세히 묘사해 주시겠어요?
    바다가 보이는 작은 카페에 앉아 있는 모습이 떠올라요. 하얀 테이블 위에 아이스 아메리카노가 놓여있고, 창문 너머로 푸른 바다와 하늘이 맞닿아 있어요. 바닷가에는 몇몇 사람들이 산책하고 있고, 파도 소리가 은은하게 들려요.
    
    만약 친구나 가족에게 이 장소(또는 대상)를 소개한다면 어떤 부분을 가장 강조하고 싶은가요?
    바다를 바라보며 여유롭게 커피를 마실 수 있는 분위기를 강조하고 싶어요. 특히 해질녘에는 노을이 바다에 반사되어 정말 아름다운 풍경을 볼 수 있어요. 또한 카페의 디저트도 맛있고, 사장님이 친절하셔서 편안한 시간을 보낼 수 있어요.
    
    그 대상이나 장소를 한 마디로 표현한다면 무엇이라고 표현할 수 있을까요? 이유도 알려주세요.
    '힐링'이라고 표현할 수 있을 것 같아요. 일상에서 벗어나 잠시 바다를 바라보며 커피 한 잔 마시는 시간이 마음의 평화를 가져다 주고 재충전할 수 있게 해주기 때문이에요.
    




> Entering new AgentExecutor chain...
Thought: 사용자의 묘사 질문에 대한 답변을 바탕으로 OPIC IH 수준의 스크립트를 생성해야 합니다. 사용자의 답변을 분석하여 스크립트 생성에 필요한 정보를 추출합니다.
Action:
```
{
  "action": "script_generator",
  "action_input": {
    "image_description": "바다가 보이는 작은 카페에 앉아 있는 모습이 떠올라요. 하얀 테이블 위에 아이스 아메리카노가 놓여있고, 창문 너머로 푸른 바다와 하늘이 맞닿아 있어요. 바닷가에는 몇몇 사람들이 산책하고 있고, 파도 소리가 은은하게 들려요.",
    "highlight_point": "바다를 바라보며 여유롭게 커피를 마실 수 있는 분위기를 강조하고 싶어요. 특히 해질녘에는 노을이